In [148]:
import pandas as pd
import spacy
import pdb
from nltk.corpus import stopwords

In [152]:
list_stopWords=list(set(stopwords.words('english')))

In [4]:
nlp = spacy.load('en_core_web_md')

In [5]:
df_demo = pd.read_csv('demo.csv',index_col=0)

In [6]:
df_demo.head()

,class,date,description,title
0,Disasters and accidents,2017/9/1,"Hurricane Irma, now a Category 2 hurricane wit...",2017 Atlantic hurricane season
1,Health and medicine,2017/9/1,"Researchers report, in the Environmental Scien...",Great Lakes
2,International relations,2017/9/1,South Korean President Moon Jae-in and U.S. Pr...,South Korea鈥揢nited States relations
3,Law and crime,2017/9/1,United States federal judge Richard Posner has...,United States Court of Appeals for the Seventh...
4,Politics and elections,2017/9/1,"The Labour Party, led by Jacinda Ardern, surge...","New Zealand general election, 2017"


In [7]:
def class_code(type_str):
    type_str = type_str.lower()
    if 'armed' in type_str or 'attack' in type_str or 'conflict' in type_str:
        return 1
    elif 'disaster' in type_str or 'accident' in type_str:
        return 2
    elif 'law' in type_str or 'crime' in type_str:
        return 3
    elif 'politic' in type_str or 'election' in type_str:
        return 4
    elif 'international' in type_str or 'relation' in type_str:
        return 5
    elif 'science' in type_str or 'technology' in type_str:
        return 6
    elif 'business' in type_str or 'econom' in type_str:
        return 7
    elif 'art' in type_str or 'culture' in type_str:
        return 8
    elif 'sport' in type_str:
        return 9
    elif 'health' in type_str or 'environment' in type_str:
        return 10
    else:
        return 0

In [8]:
df_demo['class_code'] = df_demo['class'].apply(class_code)

In [9]:
df_demo.head()

,class,date,description,title,class_code
0,Disasters and accidents,2017/9/1,"Hurricane Irma, now a Category 2 hurricane wit...",2017 Atlantic hurricane season,2
1,Health and medicine,2017/9/1,"Researchers report, in the Environmental Scien...",Great Lakes,10
2,International relations,2017/9/1,South Korean President Moon Jae-in and U.S. Pr...,South Korea鈥揢nited States relations,5
3,Law and crime,2017/9/1,United States federal judge Richard Posner has...,United States Court of Appeals for the Seventh...,3
4,Politics and elections,2017/9/1,"The Labour Party, led by Jacinda Ardern, surge...","New Zealand general election, 2017",4


In [10]:
def description_clean(description):
    description = description.split('. (')[0]+'.'
    return description

In [11]:
df_demo['des_clean'] = df_demo['description'].apply(description_clean)

In [12]:
from spacy import displacy
from IPython.display import display, HTML

In [13]:
def class_similarity(class_text,span):
    return nlp(class_text).similarity(nlp(span))

In [14]:
def print_pos_(doc,class_text):
    doc_pos_ = []
    for token in doc:
        if token.pos_ in ['VERB']:   #['AUX','NOUN','PROPN','VERB']:
            doc_pos_.extend([token.text,token.pos_,str(class_similarity(class_text,token.text))])
    print('|'.join(doc_pos_))

In [15]:
def print_noun_chunks(doc,class_text):
    noun_chunks =[]
    for i in doc.noun_chunks:
        noun_chunks.extend([i.text,str(class_similarity(class_text,i.text))])
    print('|'.join(noun_chunks))

In [16]:
def expand_with_noun_chuncks(noun_chuncks,ent):
    for noun_chunck in noun_chuncks:
        if ent in noun_chunck:
            return noun_chunck
    return ent

In [32]:
def argument_candidate(doc,class_text):
    arguments = []
    noun_chuncks = [i.text for i in doc.noun_chunks]
    for i in doc.ents:
        #arguments.append((expand_with_noun_chuncks(noun_chuncks,i.text),i.label_,str(class_similarity(class_text,expand_with_noun_chuncks(noun_chuncks,i.text)))))
        arguments.append((i.text,i.label_))
    related_when = [i for i in arguments if i[1] in ['TIME','DATE']]
    related_where = [i for i in arguments if i[1] in ['GPE','LOC','FACILITY']]
    related_who = [i for i in arguments if i[1] in ['PERSON','NORP','ORG','']]
    related_what = [i for i in arguments if i[1] in ['PRODUCT','EVENT','WORK_OF_ART','LAW','LANGUAGE','PERCENT','MONEY','QUANTITY','ORDINAL','CARDINAL']]
    return arguments,{
        'related_when':related_when,
        'related_where':related_where,
        'related_who':related_who,
        'related_what':related_what,
    }

In [84]:
def trigger_candidate(doc,class_text,doc_index):
    triggers = []
    for token in doc:
        #if not token.is_stop and token.tag_.startswith('NN') or token.tag_.startswith('V'):
        if not token.is_stop and token.tag_.startswith('V'):
            if token.text.lower() in EFITF[doc_index].keys():
                triggers.append((token.text,token.tag_,str(class_similarity(class_text,token.text)+EFITF[doc_index][token.text.lower()])))
    triggers = sorted(triggers,key=lambda x:x[2],reverse=True)[:3]
    return triggers

In [85]:
def print_arguments_triggers(arguments_dict,triggers):
    print('Arguments:')
    #pdb.set_trace()
    for k,v in arguments_dict.items():
        print(k+' : '+'|'+'|'.join(['|'.join(i) for i in v]))
    triggers_str = ''
    for i in triggers:
        triggers_str+='|'+'|'.join(i)
    print('Triggers:',triggers_str)   

In [174]:
def get_kw_set(doc,class_text,doc_index):
    kws = []
    for i in doc.ents:
        #kws.extend(i.text.split(' '))
        kws.append(i.text)
    triggers = []
    for token in doc:
        if not token.is_stop and token.tag_.startswith('V'):
            if token.text.lower() in EFITF[doc_index].keys():
                triggers.append((token.text,token.tag_,str(class_similarity(class_text,token.text)+EFITF[doc_index][token.text.lower()])))
    triggers = sorted(triggers,key=lambda x:x[2],reverse=True)[:3]
    for i in triggers:
        kws.append(i[0])
    noun_chunks = []
    for i in doc.noun_chunks:
        noun_chunks.append((i.text,str(class_similarity(class_text,i.text))))
    #noun_chunks = sorted(noun_chunks,key=lambda x:x[1],reverse=True)[:1]
    #for i in noun_chunks:
     #   kws.extend(i[0].split(' '))
    kws.append(sorted(noun_chunks,key=lambda x:x[1],reverse=True)[0][0].split(' ')[-1])
    kws = [w for w in kws if not w in list_stopWords]
    return list(set(kws))

In [178]:
def print_kws_(kws):
    kws_str = ''
    for i in kws:
        kws_str+='|'+i
    print('kws:',str(len(kws)),kws_str)  

In [179]:
def display_doc(doc,style='ent',class_text=None,doc_index=None):
    doc = nlp(doc)
    display(HTML(displacy.render(doc, style=style)))
    arguments,arguments_dict = argument_candidate(doc,class_text)
    triggers = trigger_candidate(doc,class_text,doc_index)
    print_arguments_triggers(arguments_dict,triggers)
    #print_pos_(doc,class_text)
    print_noun_chunks(doc,class_text)
    kws  = get_kw_set(doc,class_text,doc_index)
    print_kws_(kws)

In [180]:
for event in df_demo.iterrows():
    print(event[0])
    print(event[1]['class'],'\t',event[1]['date'],'\t',event[1]['title'])
    display_doc(event[1]['des_clean'],class_text=event[1]['class'].replace('and',''),doc_index=event[0])
    

0
Disasters and accidents 	 2017/9/1 	 2017 Atlantic hurricane season


Arguments:
related_when : |this weekend|DATE|next Thursday|DATE
related_where : |the Leeward Islands|LOC|Caribbean|LOC
related_who : |Hurricane Irma|PERSON
related_what : |Category 2|QUANTITY|110 miles per hour|QUANTITY|175 kilometers per hour|QUANTITY
Triggers: |increasing|VBG|0.576451752579|expected|VBN|0.476465759179|strengthening|VBG|0.427861711087|resume|VB|0.36165819714|nears|VBZ|0.354343659864
Hurricane Irma|0.357908785797|now a Category 2 hurricane|0.380235448992|maximum sustained winds|0.374320373452|110 miles|0.13337395046|hour|0.162428970276|175 kilometers|0.0666239555169|hour|0.162428970276|the danger|0.462647103084|it|0.264386930191|the Leeward Islands|0.247873127015|the Caribbean|0.208435926875
kws: 12 |175 kilometers per hour|Category 2|strengthening|danger|this weekend|Caribbean|increasing|next Thursday|the Leeward Islands|Hurricane Irma|expected|110 miles per hour
1
Health and medicine 	 2017/9/1 	 Great Lakes


Arguments:
related_when : |
related_where : |the Niagara River|LOC|the Great Lakes|LOC|Lakes Erie|LOC|Ontario|GPE
related_who : |the Environmental Science & Technology|ORG
related_what : |10|CARDINAL
Triggers: |include|VBP|0.516214151193|report|VBP|0.511450318452|introduced|VBN|0.438623985499|said|VBN|0.42566708155|links|VBZ|0.425454435351
Researchers|0.486778847535|the Environmental Science & Technology journal|0.626985301713|the discovery|0.409114996831|antidepressant concentrations|0.621126117878|10 kinds|0.338455982602|fish|0.307684981229|the Niagara River|0.388616151142|the Great Lakes|0.358734376058|Lakes Erie|0.32894182904|Ontario|0.32870366923|Speculated causes|0.402148572707|inadequate treatment|0.606032998365|water|0.354915577487|said bodies|0.362389242075|water|0.354915577487
kws: 10 |report|the Environmental Science & Technology|include|the Niagara River|Ontario|10|the Great Lakes|Lakes Erie|introduced|journal
2
International relations 	 2017/9/1 	 South Korea鈥揢nited States

Arguments:
related_when : |
related_where : |U.S.|GPE|South Korea|GPE|South Korea's|GPE
related_who : |South Korean|NORP|Moon Jae|PERSON|Donald Trump|PERSON
related_what : |
Triggers: |revise|VB|0.53351519271|agree|VBP|0.503212040706|caps|VBZ|0.317745383302
South Korean President Moon Jae-in|0.40294889442|U.S. President Donald Trump|0.413748916657|the South Korea Ballistic Missile Range Guidelines|0.486171632835|South Korea's missile development|0.51063238924
kws: 10 |South Korea's|agree|South Korean|Donald Trump|Moon Jae|South Korea|U.S.|development|revise|caps
3
Law and crime 	 2017/9/1 	 United States Court of Appeals for the Seventh Circuit


Arguments:
related_when : |three decades|DATE|1981|DATE
related_where : |United States|GPE|Chicago|GPE|Illinois|GPE
related_who : |Richard Posner|PERSON|U.S. Circuit Court of Appeals|ORG|Ronald Reagan|PERSON|Posner|PERSON|bench.(AP|ORG
related_what : |7th|ORDINAL|more than 3,300|CARDINAL
Triggers: |written|VBN|0.547143896013|Appointed|VBN|0.456781502092|headquartered|VBN|0.414551141092|announced|VBN|0.369286792484
United States federal judge Richard Posner|0.631050195264|his retirement|0.423290169946|three decades|0.423120644064|service|0.284943002868|the bench|0.343527492927|the 7th U.S. Circuit Court|0.529322510022|Appeals|0.462955921122|Chicago|0.29550988158|Illinois|0.342008780656|President Ronald Reagan|0.265534860052|Justice Posner|0.45301231031|more than 3,300 opinions|0.398155457616|the bench.(AP|0.403803427413
kws: 15 |1981|three decades|Appointed|Illinois|7th|Ronald Reagan|U.S. Circuit Court of Appeals|United States|Richard Posner|Chicago|Posner|more than 3,300|written|headqu

...

Arguments:
related_when : |
related_where : |Bengaluru|GPE
related_who : |Indian|NORP|Lankesh Patrike|PERSON|Gauri Lankesh|PERSON
related_what : |
Triggers: |shot|VBN|0.515839042646
Prominent Indian journalist|0.42397370554|Lankesh Patrike editor|0.17564787189|Gauri Lankesh|0.0|unidentified men|0.388254146727|her house|0.409643719279|the city|0.500354653334|Bengaluru|0.0651254167868
kws: 6 |Gauri Lankesh|city|Indian|Bengaluru|shot|Lankesh Patrike
33
Law and crime 	 2017/9/5 	 Australian Marriage Law Postal Survey (Same-sex marriage)


Arguments:
related_when : |
related_where : |
related_who : |The High Court of Australia|ORG|Australian|NORP
related_what : |(The Guardian)|WORK_OF_ART
Triggers: |citing|VBG|0.572195904063|receives|VBZ|0.425439919036
The High Court|0.593857575155|Australia|0.25587200476|a complaint|0.515442505477|the Australian Marriage Law Postal Survey|0.711241499187|the postal survey|0.453356289133|(The Guardian|0.413507017033
kws: 6 |The High Court of Australia|citing|Survey|Australian|receives|(The Guardian)
34
Politics and elections 	 2017/9/5 	 Politics of Taiwan


Arguments:
related_when : |
related_where : |Tainan|GPE|the Republic of China|GPE
related_who : |William Lai Ching-te|PERSON|Tsai Ing-wen|PERSON
related_what : |
Triggers: |appointed|VBN|0.551664357862
Tainan Mayor William Lai Ching-te|0.269547018921|the new Premier|0.332105353284|the President|0.521588055935|the Republic|0.534595955881|China|0.156207825947|Tsai Ing-wen|0.1266557292
kws: 6 |Tsai Ing-wen|appointed|the Republic of China|Republic|Tainan|William Lai Ching-te
35
Politics and elections 	 2017/9/5 	 Deferred Action for Childhood Arrivals


Arguments:
related_when : |the next six months|DATE
related_where : |U.S.|GPE
related_who : |The Trump Administration|ORG|DACA|ORG|Congress|ORG
related_what : |about 800,000|CARDINAL
Triggers: |brought|VBN|0.551158832215|ending|VBG|0.533632073921|halted|VBN|0.500590097025|correct|VB|0.484616554518|called|VBN|0.478593183908
The Trump Administration|0.591466005224|the next six months|0.338907360369|it|0.359504523576|the DACA program|0.280110416205|the deportation|0.395922391098|about 800,000 people|0.327572046668|who|0.298166567757|the U.S.|0.437061635613|children|0.219449375835|Congress|0.57855267145|legislation|0.497035700057|the situation|0.45906628286
kws: 10 |The Trump Administration|halted|Congress|DACA|brought|ending|about 800,000|the next six months|Administration|U.S.
36
Sports 	 2017/9/5 	 2017鈥?8 NBA season


Arguments:
related_when : |
related_where : |
related_who : |Tilman Fertitta|PERSON|Houston Rockets NBA|ORG
related_what : |US$2.2 billion|MONEY
Triggers: |buys|VBZ|0.472414864905
Billionaire investor Tilman Fertitta|0.11165074302|the Houston Rockets NBA basketball team|0.599674816765
kws: 5 |buys|team|US$2.2 billion|Houston Rockets NBA|Tilman Fertitta


In [181]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from collections import defaultdict

In [97]:
def efitf(X):
    count = CountVectorizer(stop_words='english')
    X_train_count = count.fit_transform(X)
    tfidf = TfidfTransformer(use_idf=True,smooth_idf=True,sublinear_tf=True)
    X_train_tfidf = tfidf.fit_transform(X_train_count)
    tf_feature_names = count.get_feature_names()
    X_train_tfidf = [list(i) for i in list(X_train_tfidf.toarray())]
    EFITF = defaultdict(dict)
    for Type,values in enumerate(X_train_tfidf):
        for index,value in enumerate(values):
            if value > 0.0:
                EFITF[Type].update({tf_feature_names[index]:value}) 
    return EFITF

In [98]:
X = []

In [99]:
X = df_demo['des_clean'].tolist()

In [100]:
EFITF = efitf(X)

In [104]:
'demo'

['demo']

In [144]:
a = [1,2,3,4,5]

In [146]:
a

[1, 3, 4, 5]

In [167]:
doc = nlp('arrested')

In [168]:
doc[0].pos_,doc[0].tag_

('VERB', 'VBN')